# Data pre-processing

__Goal:__ Build dataset for training and evaluation relation extraction models

__Method:__ Extract relation data from manually curated covid-19 kg, split into training and test sets

__Data:__ [Covid-19 KG](https://github.com/covid19kg/covid19kg) from Fraunhofer Institute for Algorithms and Scientific Computing SCAI

__Tools:__ PyBEL

__Result:__ covid-19-kg dataset

In [1]:
import requests
import pybel
import pandas as pd

In [2]:
#load graph
url = 'https://github.com/covid19kg/covid19kg/raw/master/covid19kg/_cache.bel.nodelink.json'
res = requests.get(url)
graph = pybel.from_nodelink(res.json())

In [3]:
#export  graph as a tab-separated edge list.
pybel.to_csv(graph, 'covid19kg_dataset.csv')

In [7]:
#load csv
dataset = pd.read_csv('covid19kg_dataset.csv', sep='\t')

In [8]:
dataset.head()

,"a(CHEBI:""(+)-Tetrandrine"")",negativeCorrelation,act(p(MGI:Tpcn2)),"{""annotations"": {""PublicationStatus"": {""Published"": true}, ""PublicationType"": {""Research"": true}, ""Section"": {""Results"": true}}, ""citation"": {""authors"": [""Chen J"", ""Chen T"", ""Chen X"", ""Guo L"", ""Guo R"", ""Hu J"", ""Hu K"", ""Jin Q"", ""Lei X"", ""Li P"", ""Liu Y"", ""Mi D"", ""Mu Z"", ""Ou X"", ""Qian Z"", ""Ren L"", ""Wang J"", ""Xiang Z""], ""date"": ""2020-03-27"", ""db"": ""PubMed"", ""db_id"": ""32221306"", ""db_name"": ""Nature communications"", ""first"": ""Ou X"", ""last"": ""Qian Z"", ""pages"": ""1620"", ""volume"": ""11""}, ""evidence"": ""While blocking TPC2 activity by tetrandrine, an inhibitor for TPC237, decreased entry of SARS-CoV-2 S pseudovirions (Fig. 3f), treatment of cells with 130, a TRPML1 inhibitor, had no effect (Supplementary Fig. 1), indicating that TPC2, not TRPML1, is important for SARS-CoV-2 entry."", ""line"": 96, ""object"": {""modifier"": ""Activity""}, ""relation"": ""negativeCorrelation""}"
0,"a(CHEBI:""(+)-Tetrandrine"")",negativeCorrelation,"a(TAX:""Severe acute respiratory syndrome coron...","{""annotations"": {""PublicationStatus"": {""Publis..."
1,"a(CHEBI:""(S)-verapamil"")",association,p(FIXME:orf9c),"{""annotations"": {""Experiment"": {""Literature de..."
2,"a(CHEBI:""(S)-verapamil"")",negativeCorrelation,path(DO:hypertension),"{""annotations"": {""Experiment"": {""Literature de..."
3,"a(CHEBI:""1-phenyl-1-cyclopentanecarboxylic aci...",partOf,"complex(a(CHEBI:""1-phenyl-1-cyclopentanecarbox...","{""relation"": ""partOf""}"
4,"a(CHEBI:""3,3',5'-triiodothyronine"")",negativeCorrelation,"path(DO:""COVID-19"")","{""annotations"": {""Cohort_Median_Age"": {""62 yea..."
